In [12]:
#without tkinter
# !pip install SpeechRecognition
# !pip install moviepy
# !pip install googletrans
# !pip install vaderSentiment
#!pip install deepface
# !pip install cv
# !pip install tkinter
#!pip install --upgrade googletrans==4.0.0-rc1
#!pip install translate

In [13]:
import json

def read_question_times(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    
    question_times = []
    for item in data[1:]:  # Exclude the first question
        question_times.append(item['timeInSeconds'])
    
    return question_times

# Example usage:
filename = 'Questions.json'
question_times = read_question_times(filename)
print(question_times)


[3, 7, 4, 6, 4, 2, 4, 3, 5, 2]


In [14]:
def calculate_percentage_frequency(array):
    frequency = {}
    total_entries = len(array)
    for item in array:
        frequency[item] = frequency.get(item, 0) + 1

    percentage_frequency = {key: (value / total_entries) * 100 for key, value in frequency.items()}
    return percentage_frequency

# Example usage:
data = [1, 2, 2, 3, 3, 3, 4, 4, 4, 4]
result = calculate_percentage_frequency(data)
print(result)


{1: 10.0, 2: 20.0, 3: 30.0, 4: 40.0}


In [15]:
import random

# Generate an array with 50 random values
random_values = [random.randint(0, 100) for _ in range(51)]

# Array A with specified lengths for rows
A = [5, 6, 7, 6, 9, 8, 4, 2, 2, 2]

# Initialize an empty 2D array
array_2d = []

# Counter to keep track of the index in array A
index = 0

# Loop to generate each row of the 2D array
for row_length in A:
    row = random_values[index:index+row_length]
    array_2d.append(row)
    index += row_length

# Print the 2D array
for row in array_2d:
    print(row)


[47, 91, 1, 63, 29]
[53, 32, 68, 66, 40, 22]
[64, 77, 47, 99, 27, 28, 54]
[13, 25, 10, 98, 51, 37]
[27, 37, 17, 25, 44, 100, 90, 82, 26]
[48, 94, 10, 37, 36, 12, 88, 80]
[99, 13, 70, 4]
[74, 38]
[90, 94]
[87, 89]


### Just Run This

In [1]:
import speech_recognition as sr
from translate import Translator as TranslateTranslator
from moviepy.editor import VideoFileClip
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deepface import DeepFace
import cv2
import json
import csv
import random

# Function to store emotions in the csv file
def create_csv(emotion_dict, question_number, patient_id, heart_rate, blood_oxygen_level, sentiment):
    emotions = ['happy', 'sad', 'angry', 'neutral', 'fear', 'surprise', 'disgust']
    percentages = {emotion: 0.0 for emotion in emotions}
    
    for emotion, percentage in emotion_dict.items():
        percentages[emotion] = percentage
    
    with open('emotions.csv', mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['Patient ID', 'Heart Rate', 'Blood Oxygen Level', 'Sentiment', 'Question'] + emotions)
        if file.tell() == 0:
            writer.writeheader()
        percentages['Question'] = f'q{question_number}'
        percentages['Patient ID'] = patient_id
        percentages['Heart Rate'] = heart_rate
        percentages['Blood Oxygen Level'] = blood_oxygen_level
        percentages['Sentiment'] = sentiment
        writer.writerow(percentages)

#extract question time from json file
def read_question_times(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    
    question_times = []
    for item in data[1:]:  # Exclude the first question
        question_times.append(item['timeInSeconds'])
    
    return question_times

# function to calculate percentage frequency of emotions
def calculate_percentage_frequency(array):
    frequency = {}
    total_entries = len(array)
    for item in array:
        frequency[item] = frequency.get(item, 0) + 1

    percentage_frequency = {key: (value / total_entries) * 100 for key, value in frequency.items()}
    return percentage_frequency

# Function to extract audio and save as WAV
def extract_audio_and_save_as_wav(input_video_path, output_audio_path):
    video_clip = VideoFileClip(input_video_path)
    audio_clip = video_clip.audio

    audio_clip.write_audiofile(output_audio_path, codec='pcm_s16le', fps=44100)

    video_clip.close()

# Function to recognize Urdu audio
def recognize_urdu_audio(audio_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language="ur-PK")  # Recognizing Urdu
        return text
    except sr.UnknownValueError:
        return "Google Web Speech API could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results from Google Web Speech API; {e}"
    
# Function to generate a random number for heart rate
def generate_random_heart_rate():
    return random.randint(60, 100)

# Function to generate a random number for blood oxygen level
def generate_random_blood_oxygen_level():
    return random.randint(95, 100)

# Function to generate a random 3 digit number for patient ID
def generate_random_patient_id():
    return random.randint(100, 999)


# Function to translate Urdu text to English
def translate_urdu_to_english(urdu_text):
    translator = TranslateTranslator(to_lang="en", from_lang="ur")
    translation = translator.translate(urdu_text)
    return translation

# Function for sentiment analysis
def analyze_sentiment(english_text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(english_text)
    
    # Interpret the compound score
    if sentiment_scores['compound'] >= 0.05:
        return "Positive"
    elif sentiment_scores['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Main execution
if __name__ == "__main__":
    # Replace with the path to your MP4 file
    input_video_path = "C:\\Users\\Sys\\Desktop\\model\\demo.mp4"
    
    # Replace with the desired output path and filename for the WAV file
    output_audio_path = "C:\\Users\\Sys\\Desktop\\model\\output_audio.wav"

    

    q1Emotions = []
    q2Emotions = []
    q3Emotions = []
    q4Emotions = []
    q5Emotions = []
    q6Emotions = []
    q7Emotions = []
    q8Emotions = []
    q9Emotions = []
    q10Emotions = []

    # Extract audio from video
    extract_audio_and_save_as_wav(input_video_path, output_audio_path)

    # Recognize Urdu audio
    urdu_text = recognize_urdu_audio(output_audio_path)
    

    # Translate Urdu text to English
    english_text = translate_urdu_to_english(urdu_text)
    

    # Analyze sentiment of English text
    sentiment = analyze_sentiment(english_text)
    

    # Open a video capture object
    cap = cv2.VideoCapture(input_video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

##########################################################################################

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # calculate frames per second of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # print the total number of frames and frames per second
    print(f"Total Frames: {total_frames}, Frames per Second: {fps}")

    filename = 'Questions.json'
    question_times = read_question_times(filename)

    # Process one frame per second
    frame_num = 0

    # store each frame number in an array
    frame_numbers = []
    while frame_num < total_frames:
        frame_numbers.append(frame_num)
        frame_num += int(fps)

    frame_allotted_to_each_question = []

    # Counter to keep track of the index in array A
    index = 0

    # Loop to generate each row of the 2D array
    for row_length in question_times:
        row = frame_numbers[index:index+row_length]
        frame_allotted_to_each_question.append(row)
        index += row_length


    # Print the 2D array
    for row in frame_allotted_to_each_question:
        print(row)

##########################################################################################
    frame_num = 0
    all_emotions = []
    while frame_num < total_frames:
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

        # Read the selected frame
        ret, frame = cap.read()

        # Check if the frame was read successfully
        if not ret:
            print(f"Error reading frame {frame_num}")
            continue

        # Analyze the emotion in the frame
        result = DeepFace.analyze(frame, actions=['emotion'])

        # Extract the dominant emotion
        dominant_emotion = result[0]['dominant_emotion']

        # store the emotion in the array
        all_emotions.append(dominant_emotion)

        # Print the result to the console
        #print(f"Frame {frame_num/30}: Dominant Emotion - {dominant_emotion}")

        # Increment frame number by FPS to process one frame per second
        frame_num += int(fps)
##########################################################################################
        
    #store questionwise emotions in a 2D array
    index = 0
    questionwise_emotions = []
    for row_length in question_times:
        row = all_emotions[index:index+row_length]
        questionwise_emotions.append(row)
        index += row_length
        
##########################################################################################
        
    question1_emotions = questionwise_emotions[0]
    question2_emotions = questionwise_emotions[1]
    question3_emotions = questionwise_emotions[2]
    question4_emotions = questionwise_emotions[3]
    question5_emotions = questionwise_emotions[4]
    question6_emotions = questionwise_emotions[5]
    question7_emotions = questionwise_emotions[6]
    question8_emotions = questionwise_emotions[7]
    question9_emotions = questionwise_emotions[8]
    question10_emotions = questionwise_emotions[9]

##########################################################################################

    # Release the video capture object
    print(all_emotions)

    #for row in questionwise_emotions:
    #    print(row)

    q1EmotionsPercetage = calculate_percentage_frequency(question1_emotions)
    q2EmotionsPercetage = calculate_percentage_frequency(question2_emotions)
    q3EmotionsPercetage = calculate_percentage_frequency(question3_emotions)
    q4EmotionsPercetage = calculate_percentage_frequency(question4_emotions)
    q5EmotionsPercetage = calculate_percentage_frequency(question5_emotions)
    q6EmotionsPercetage = calculate_percentage_frequency(question6_emotions)
    q7EmotionsPercetage = calculate_percentage_frequency(question7_emotions)
    q8EmotionsPercetage = calculate_percentage_frequency(question8_emotions)
    q9EmotionsPercetage = calculate_percentage_frequency(question9_emotions)
    q10EmotionsPercetage = calculate_percentage_frequency(question10_emotions)
    

    emotions_dicts = [q1EmotionsPercetage, q2EmotionsPercetage, q3EmotionsPercetage, q4EmotionsPercetage,
                  q5EmotionsPercetage, q6EmotionsPercetage, q7EmotionsPercetage, q8EmotionsPercetage,
                  q9EmotionsPercetage, q10EmotionsPercetage]
    
    
    # generate random number for patient ID
    patient_id = generate_random_patient_id()

    # store data in csv file
    for i in range(10):
        heart_rate = generate_random_heart_rate()
        blood_oxygen_level = generate_random_blood_oxygen_level()
        create_csv(emotions_dicts[i], i+1, patient_id, heart_rate, blood_oxygen_level, sentiment)
        
    cap.release()


MoviePy - Writing audio in C:\Users\Sys\Desktop\model\output_audio.wav


MoviePy - Done.
Total Frames: 1200, Frames per Second: 30.0
[0, 30, 60]
[90, 120, 150, 180, 210, 240, 270]
[300, 330, 360, 390]
[420, 450, 480, 510, 540, 570]
[600, 630, 660, 690]
[720, 750]
[780, 810, 840, 870]
[900, 930, 960]
[990, 1020, 1050, 1080, 1110]
[1140, 1170]
['angry', 'angry', 'angry', 'angry', 'angry', 'happy', 'happy', 'happy', 'happy', 'happy', 'sad', 'sad', 'sad', 'sad', 'sad', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'fear', 'fear', 'fear', 'fear', 'fear', 'happy', 'happy', 'happy', 'happy', 'happy', 'angry', 'angry', 'angry', 'angry', 'angry', 'sad', 'sad', 'sad', 'sad', 'sad']


# What to do next?

### make the changes in the server according to the changes made above
### firebase waala kaam tu kr, mujhay nai aata
### meinay frontend pe bhi kaam krna